<a href="https://colab.research.google.com/github/raja-jaloka/LOCALREPO/blob/main/Spam_Email_Project(CPC)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CPC PROJECT-1 SPAM EMAIL DETECTION/CLASSIFIER

Loading & Understanding the Data

In [ ]:
#Step 1: Loading Data
import pandas as pd #will be used in this project to access the csv dataset used to train our model
import numpy as np #will be used for array operations on the accessed dataset
from google.colab import files #one of the multiple methods to upload a file saved on the local system onto the google collab notebook
uploaded=files.upload() #Notice a pop-up opens when this line is run asking to select the file
import io #input output library
data=pd.read_csv(io.BytesIO(uploaded['spam_ham_dataset (1).csv']), encoding='latin-1') #uploaded[...] is a dictionary of the raw binary code which using the io.BytesIO is converted to be read in a file format but we faced a problem here because of certain encodings in the file so we used the encoding='latin-1'

Saving spam_ham_dataset.csv to spam_ham_dataset (1).csv


In [ ]:
#Understanding Data
print(data.columns)
print(data.info)
print(data.head())

Index(['Unnamed: 0', 'label', 'text', 'label_num'], dtype='object')
<bound method DataFrame.info of       Unnamed: 0 label                                               text  \
0            605   ham  Subject: enron methanol ; meter # : 988291\nth...   
1           2349   ham  Subject: hpl nom for january 9 , 2001\n( see a...   
2           3624   ham  Subject: neon retreat\nho ho ho , we ' re arou...   
3           4685  spam  Subject: photoshop , windows , office . cheap ...   
4           2030   ham  Subject: re : indian springs\nthis deal is to ...   
...          ...   ...                                                ...   
5166        1518   ham  Subject: put the 10 on the ft\nthe transport v...   
5167         404   ham  Subject: 3 / 4 / 2000 and following noms\nhpl ...   
5168        2933   ham  Subject: calpine daily gas nomination\n>\n>\nj...   
5169        1409   ham  Subject: industrial worksheets for august 2000...   
5170        4807  spam  Subject: important online ban

Data Cleaning

In [ ]:
#Cleaning the data
'''from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='mean')
imputed_data=imputer.fit_transform(data)
print(data)'''
#Updates 4/10/25 : currently working on finding another method to correct the ValueError:Cannot use mean strategy with non-numeric data:could not convert string to float: 'ham'
#Updates 4/10/25: Learning more about using OneHotEncoder instead
data.duplicated() #Therefore all the rows are unique

,0
0,False
1,False
2,False
3,False
4,False
...,...
5166,False
5167,False
5168,False
5169,False


In [ ]:
#now we segregate the columns into numerical and categorical types---dssh.csv(our training dataset) won't specifically require for this segregation this however is done keeping in mind all sorts of datasets
cat_col=[col for col in data.columns if data[col].dtype=='object']
num_col=[col for col in data.columns if data[col].dtype!='object']
print(cat_col)
print(num_col)

['label', 'text']
['Unnamed: 0', 'label_num']


In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
#now we access the text and perform removing stopwords, punctuations & normalization of the data
from nltk.tokenize import word_tokenize
data['tokens']=data['text'].apply(word_tokenize) #.apply tokenizes each row of text
#print(data['tokens']) #successfully tokenized

#now we normalize the data
import string
data['tokens'] = [[str(tok).lower() for tok in token_list] for token_list in data['tokens']]
#print(data['tokens']) #normalizing is to convert all characters in the token_list to lowercase

#now we remove punctuations
import string
data['tokens'] = [[tok for tok in token_list if tok not in string.punctuation] for token_list in data['tokens']]
#print(data['tokens'])

#now we remove stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=set(stopwords.words('english')) #initialised stop_word to a set of english words in stopword. set was used since set has a fundamental property of not containing duplicate entries
data['tokens']=[[token for token in token_list if token not in stop_words]for token_list in data['tokens']]
print(data['tokens'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0       [subject, enron, methanol, meter, 988291, foll...
1       [subject, hpl, nom, january, 9, 2001, see, att...
2       [subject, neon, retreat, ho, ho, ho, around, w...
3       [subject, photoshop, windows, office, cheap, m...
4       [subject, indian, springs, deal, book, teco, p...
                              ...                        
5166    [subject, put, 10, ft, transport, volumes, dec...
5167    [subject, 3, 4, 2000, following, noms, hpl, ta...
5168    [subject, calpine, daily, gas, nomination, jul...
5169    [subject, industrial, worksheets, august, 2000...
5170    [subject, important, online, banking, alert, d...
Name: tokens, Length: 5171, dtype: object


In [ ]:
#LEMMATIZATION OF THE DATA
'''We used Lemmatization here because lemmatization converts a token into it's
grammatically correct lemma/reduced form'''
import spacy
nlp=spacy.load('en_core_web_sm')
data["tokens"]=[[token.lemma_ for token in nlp(' '.join(token_list))] for token_list in data['tokens']]
'''when you process text with spacy by passing the npl(text) function
the text is tokenized creating a doc object which is passed through several
pipeline components which include:
tokenizer
tagger=assigns part of speech tags to tokens
lemmatizer=converts tokens to their base forms
parser=analyzes grammatical dependencies
entity recognizer=identifies named entities(like people or places)
Text categorizer=assigns a label to the text
custom components= user defined functions for extra processing
'''
print(data['tokens'])

0       [subject, enron, methanol, meter, 988291, foll...
1       [subject, hpl, nom, january, 9, 2001, see, att...
2       [subject, neon, retreat, ho, ho, ho, around, w...
3       [subject, photoshop, window, office, cheap, ma...
4       [subject, indian, springs, deal, book, teco, p...
                              ...                        
5166    [subject, put, 10, ft, transport, volume, decr...
5167    [subject, 3, 4, 2000, follow, nom, hpl, take, ...
5168    [subject, calpine, daily, gas, nomination, jul...
5169    [subject, industrial, worksheet, august, 2000,...
5170    [subject, important, online, banking, alert, d...
Name: tokens, Length: 5171, dtype: object


FEATURE EXTRACTION

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
#Now to create a single sparse matrix of all the tokens in the data[tokens]
corpus= [" ".join(tokens) for tokens in data["tokens"]]
X=vectorizer.fit_transform(corpus) #X is our ML input + produces a document-term matrix(features per document) as a sparse matrix
#.fit learns all the parameters corresponding to the data
#.fit_transform learns about all the parameters of the given input and then transforms it to a sparse matrix(to be used on training data)
#.transform directly transforms the given input to a sparse matrix (doc) based on the fit of the training data, this is used on test data
vectorizer.get_feature_names_out()
X.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

TRAIN TEST SPLIT

In [ ]:
from sklearn.model_selection import train_test_split
Y= data['label_num']
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.2,random_state=42)
# test_Size=0.2 means that 20% of the data is considered as test cases and the rest are used to train the model
#random_state=num indicates that as the code snippet as run the order of the train or test datas remain the same

MODEL TRAINING


1.   Logistic Regression




In [37]:
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(X_train,Y_train)
model1.predict(X_test)

from sklearn.metrics import accuracy_score
print("The accuracy score is: ",end=' ')
print(accuracy_score(Y_test,model1.predict(X_test)))

from sklearn.metrics import recall_score #tp/(tp+fn)
print("The recall scores are: ",end='=')
print(recall_score(Y_test,model1.predict(X_test),average='binary',pos_label=1),end='---')
print(recall_score(Y_test,model1.predict(X_test),average='binary',pos_label=0))

from sklearn.metrics import precision_score #tp/(tp+fp)
print('The precision scores are: ',end='=')
print(precision_score(Y_test,model1.predict(X_test),average='binary',pos_label=1),end='---')
print(precision_score(Y_test,model1.predict(X_test),average='binary',pos_label=0))

from sklearn.metrics import f1_score #2*tp/(2*tp+fp+fn)
print("The f1_scores are: ",end='=')
print(f1_score(Y_test,model1.predict(X_test),average='binary',pos_label=1),end='---')
print(f1_score(Y_test,model1.predict(X_test),average='binary',pos_label=0))

The accuracy score is:  0.9816425120772947
The recall scores are: =0.9795221843003413---0.9824797843665768
The precision scores are: =0.9566666666666667---0.9918367346938776
The f1_scores are: =0.9679595278246206---0.987136086662153


Deployment

In [ ]:
!pip install streamlit-jupyter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
StreamlitPatcher().jupyter()
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import pickle

In [ ]:
!pip show streamlit-jupyter

Name: streamlit-jupyter
Version: 0.3.1
Summary: Simple Python package to preview and develop streamlit apps in jupyter notebooks
Home-page: https://github.com/ddobrinskiy/streamlit-jupyter
Author: David Dobrinskiy
Author-email: david@dobrinskiy.me
License: Apache Software License 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: fastcore, ipywidgets, jupyter, stqdm, streamlit, tabulate, tqdm
Required-by: 


In [ ]:
'''from google.colab import output
output.enable_custom_widget_manager()'''

In [1]:
#THIS CODE DOESN'T RUN BECAUSE OF THE WAY STREAMLIT REACTS WITH COLAB NOTEBOOK
'''st.title(' Spam Classifier 📧')
user_input = st.text_area('Email:', height=200)
if st.button('Classify'):
    if user_input:
        input_vec = vectorizer.transform([user_input.lower()])
        pred = model1.predict(input_vec)[0]
        if pred == 1:
            st.error(' SPAM 🚫')
        else:
            st.success(' HAM ✅')
    else:
        st.warning(' Enter text')
st.caption('Spam Classifier')'''

"st.title(' Spam Classifier 📧')\nuser_input = st.text_area('Email:', height=200)\nif st.button('Classify'):\n    if user_input:\n        input_vec = vectorizer.transform([user_input.lower()])\n        pred = model1.predict(input_vec)[0]\n        if pred == 1:\n            st.error(' SPAM 🚫')\n        else:\n            st.success(' HAM ✅')\n    else:\n        st.warning(' Enter text')\nst.caption('Spam Classifier')"

In [43]:
# Example text to classify
test_email = "Congratulations! You have won a $1000 gift card. Click here to claim your prize now. Limited time offer, act fast!"

# Preprocess the text (lowercase, tokenize, remove punctuation and stopwords, lemmatize)
# We are reusing the steps of preprocessing we defined earlier
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

nltk.download('punkt', quiet=True) #quiet helps making downloading unseen
nltk.download('stopwords', quiet=True)
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [tok for tok in tokens if tok not in string.punctuation]
    tokens = [token for token in tokens if token not in stop_words]
    doc = nlp(' '.join(tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]
    return " ".join(lemmatized_tokens)

processed_email = preprocess_text(test_email)
print(f"Processed email: {processed_email}")

# Vectorize the processed text using the fitted vectorizer
input_vec = vectorizer.transform([processed_email])

# Predict the class
prediction = model1.predict(input_vec)[0]

# Print the prediction
if prediction == 1:
    print("Prediction: SPAM 🚫")
else:
    print("Prediction: HAM ✅")

Processed email: congratulation 1000 gift card click claim prize limit time offer act fast
Prediction: SPAM 🚫
